In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
conf = SparkConf()
conf.set("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")
conf.set("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf, log

In [3]:
df = spark.range(0, 10 * 1000 * 1000).withColumn('id', (col('id') / 10000).cast('integer')).withColumn('v', rand())

In [4]:

df.cache()
df.count()

10000000

In [7]:
@udf('double')
def plus_one(v):
    return v + 1

In [9]:
%time
df.withColumn('v', plus_one(df.v)).agg(count(col('v'))).show()

Wall time: 0 ns
+--------+
|count(v)|
+--------+
|10000000|
+--------+



In [11]:
df.withColumn('v', plus_one(df.v)).show()

+---+------------------+
| id|                 v|
+---+------------------+
|  0|1.0922988208363424|
|  0|1.6769451301868872|
|  0|1.3104433460112879|
|  0|1.2508636169081047|
|  0|1.9313903958594187|
|  0| 1.183732121648227|
|  0| 1.495188968500107|
|  0|1.5238617573339335|
|  0| 1.712705269168485|
|  0|1.5013970660553224|
|  0| 1.781912740230034|
|  0|1.2468175949410767|
|  0|1.7299890196075802|
|  0|1.5028392001569828|
|  0|1.5388136073133867|
|  0|1.2700669038386945|
|  0|1.8643552209460785|
|  0| 1.133052061707056|
|  0|1.5154161158068395|
|  0|1.7423036614842236|
+---+------------------+
only showing top 20 rows



In [12]:
import pandas as pd

@pandas_udf("double")
def pandas_plus_one(v: pd.Series) -> pd.Series:
    return v + 1

In [13]:
%timeit df.withColumn('v', pandas_plus_one(df.v)).agg(count(col('v'))).show()

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

+--------+
|count(v)|
+--------+
|10000000|
+--------+

3.31 s ± 69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
from scipy import stats

@udf('double')
def cdf(v):
    return float(stats.norm.cdf(v))

%timeit df.withColumn('cumulative_probability', cdf(df.v)).show()

+---+-------------------+----------------------+
| id|                  v|cumulative_probability|
+---+-------------------+----------------------+
|  0|0.09229882083634233|    0.5367696873956587|
|  0| 0.6769451301868872|    0.7507796157042829|
|  0|0.31044334601128776|    0.6218880820707676|
|  0|0.25086361690810466|    0.5990402227187244|
|  0| 0.9313903958594187|    0.8241741714617743|
|  0|0.18373212164822705|    0.5728881957857559|
|  0|0.49518896850010696|    0.6897666315801746|
|  0| 0.5238617573339335|    0.6998126522807955|
|  0|  0.712705269168485|    0.7619859223773182|
|  0| 0.5013970660553224|     0.691954147881658|
|  0| 0.7819127402300341|    0.7828670714230717|
|  0| 0.2468175949410767|      0.59747530355293|
|  0| 0.7299890196075801|    0.7673015517767563|
|  0| 0.5028392001569828|    0.6924613346944156|
|  0| 0.5388136073133867|    0.7049922632551755|
|  0|0.27006690383869447|    0.6064456083742223|
|  0| 0.8643552209460785|    0.8063036081285964|
|  0|0.1330520617070

+---+-------------------+----------------------+
| id|                  v|cumulative_probability|
+---+-------------------+----------------------+
|  0|0.09229882083634233|    0.5367696873956587|
|  0| 0.6769451301868872|    0.7507796157042829|
|  0|0.31044334601128776|    0.6218880820707676|
|  0|0.25086361690810466|    0.5990402227187244|
|  0| 0.9313903958594187|    0.8241741714617743|
|  0|0.18373212164822705|    0.5728881957857559|
|  0|0.49518896850010696|    0.6897666315801746|
|  0| 0.5238617573339335|    0.6998126522807955|
|  0|  0.712705269168485|    0.7619859223773182|
|  0| 0.5013970660553224|     0.691954147881658|
|  0| 0.7819127402300341|    0.7828670714230717|
|  0| 0.2468175949410767|      0.59747530355293|
|  0| 0.7299890196075801|    0.7673015517767563|
|  0| 0.5028392001569828|    0.6924613346944156|
|  0| 0.5388136073133867|    0.7049922632551755|
|  0|0.27006690383869447|    0.6064456083742223|
|  0| 0.8643552209460785|    0.8063036081285964|
|  0|0.1330520617070

In [16]:
from pyspark.sql import Row
@udf(ArrayType(df.schema))
def substract_mean(rows):
    vs = pd.Series([r.v for r in rows])
    vs = vs - vs.mean()
    return [Row(id=rows[i]['id'], v=float(vs[i])) for i in range(len(rows))]
  
%timeit df.groupby('id').agg(collect_list(struct(df['id'], df['v'])).alias('rows')).withColumn('new_rows', substract_mean(col('rows'))).withColumn('new_row', explode(col('new_rows'))).withColumn('id', col('new_row.id')).withColumn('v', col('new_row.v')).agg(count(col('v'))).show()

KeyboardInterrupt: 

In [17]:
# Input/output are both a pandas.DataFrame
def subtract_mean(pdf: pd.DataFrame) -> pd.DataFrame:
    return pdf.assign(v=pdf.v - pdf.v.mean())
%time
df.groupby('id').applyInPandas(subtract_mean, schema=df.schema).agg(count(col('v'))).show()

Wall time: 0 ns
+--------+
|count(v)|
+--------+
|10000000|
+--------+



In [18]:
df2 = df.withColumn('y', rand()).withColumn('x1', rand()).withColumn('x2', rand()).select('id', 'y', 'x1', 'x2')
df2.show()                                                               

+---+-------------------+-------------------+-------------------+
| id|                  y|                 x1|                 x2|
+---+-------------------+-------------------+-------------------+
|  0|0.05176543410635215|  0.806943037038435|0.29978915302427644|
|  0| 0.9823209669880942| 0.2054133829839181| 0.9256840364977199|
|  0|0.27609051528035944| 0.8400739685654518| 0.5508765390861234|
|  0|  0.486188523510939| 0.4834988250933433|0.21768758392996768|
|  0|0.17790705839223786| 0.2703754082653832|0.17122261567112307|
|  0|0.02706571104453237| 0.8373042801635908| 0.1646616526728668|
|  0| 0.8473788254153076| 0.3352634163635939| 0.6355396218493403|
|  0|0.21060665800017042| 0.9784020617235799| 0.7726625809607897|
|  0| 0.1677149104538569| 0.8684674938799263| 0.6822906604514117|
|  0| 0.8358801329232284| 0.4254381164948219| 0.5709270381213705|
|  0| 0.9042416067100224| 0.7385059064961462| 0.6433647200399656|
|  0| 0.5187631457350552|0.33869438712777333| 0.9304721963810767|
|  0| 0.98

In [19]:
import statsmodels.api as sm
# df has four columns: id, y, x1, x2
group_column = 'id'
y_column = 'y'
x_columns = ['x1', 'x2']
schema = df2.select(group_column, *x_columns).schema
# Input/output are both a pandas.DataFrame
def ols(pdf: pd.DataFrame) -> pd.DataFrame:
    group_key = pdf[group_column].iloc[0]
    y = pdf[y_column]
    X = pdf[x_columns]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return pd.DataFrame([[group_key] + [model.params[i] for i in   x_columns]], columns=[group_column] + x_columns)
beta = df2.groupby(group_column).applyInPandas(ols, schema=schema)
beta.show()

+---+--------------------+--------------------+
| id|                  x1|                  x2|
+---+--------------------+--------------------+
|148|0.002085738787324049|-0.01136035915082...|
|463|0.011114800102964728| 0.00707095867957616|
|471|0.015477085687807257|-0.00800860314279...|
|496|-0.02317909727150396|0.004937312840344436|
|833| 0.01492150034493809|-0.01152687437933...|
|243|0.006237272533267727|0.005165799267188332|
|392|0.012713518938882899|0.001132373106619...|
|540|4.902628504377393E-5|0.007437303796094524|
|623|-3.99845987331595...|-0.02022254578432...|
|737|-9.07680522716817E-4|-0.01866145335211...|
|858|   5.105755774861E-4|0.008750045132592456|
|897| 0.00798293561846123|0.006141854246674311|
| 31|-0.00524618088383...|0.003795537450328...|
|516|-0.02227875318013245|-4.99282969494837E-4|
| 85| 0.01714694711766769|0.012035540386186677|
|137|-0.01199435889867...|0.016507167582070235|
|251|0.014950289252746636|-0.00644495065150...|
|451| 0.01042210564862865|-8.37757365459